## RAG FLOW

### MinSearch

In [33]:
import minsearch
import json

In [35]:
with open('../data_gloss/kubernetes_docs_combined.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [36]:
len(docs_raw)

229

In [45]:
index = minsearch.Index(
    text_fields=["title", "text"],
    keyword_fields=["source_file"]
)

In [46]:
q = "How do I get pods status?"

In [47]:
documents = docs_raw

In [48]:
index.fit(documents)

In [49]:
from openai import OpenAI

In [50]:
client = OpenAI()

In [56]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

'To check the status of pods in a Kubernetes cluster, you can use the `kubectl` command-line tool, which is the primary interface for communicating with the Kubernetes API server.\n\nHere are steps and commands you can use to get the status of your pods:\n\n1. **Check the Status of All Pods in the Default Namespace**:\n   Run the following command to list all the pods and their statuses in the default namespace.\n\n   ```bash\n   kubectl get pods\n   ```\n\n   This command will output a list of pods with their names, readiness status, status, restarts count, and age.\n\n2. **Check the Status of All Pods in a Specific Namespace**:\n   If your pods are in a namespace other than the default, you can specify the namespace like so:\n\n   ```bash\n   kubectl get pods -n your-namespace\n   ```\n\n   Replace `your-namespace` with the name of your namespace.\n\n3. **Get Detailed Information About a Specific Pod**:\n   To get detailed information about a particular pod, use:\n\n   ```bash\n   ku

In [57]:
def search(query):
    boost = {'title': 3.0, 'source_file': 0.5}

    results = index.search(
        query=query,
        num_results=5
    )

    return results

In [58]:
def build_prompt(query, search_results):
    prompt_template = """
You are a Kubernetes assistant. Use ONLY the information in the "context" to answer the user's question.

REQUIREMENTS:
- Output ONLY raw Markdown text (no surrounding quotes, no JSON, no markdown in a string).
- Use literal line breaks for paragraphs and fenced code blocks for commands (```bash ... ```).
- Do NOT include backslash-n sequences ("\n") to indicate newlines — use real newlines.
- Do not escape code blocks or wrap them in a string.
- Return the answer only (no meta commentary).

Example of desired output:
To apply a YAML file in Kubernetes, use the following command:

```bash
kubectl apply -f FILENAME.yaml

Context:
{context}

User's Question:
{question}

Answer:
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"title: {doc['title']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [59]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [60]:
query = 'how do I get status of pods in Kubernetes?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [61]:
rag(query)

'To get the status of pods in Kubernetes, you can use the `kubectl get` command:\n\n```bash\nkubectl get pods\n```'

In [62]:
query2 = "Give me the command to apply a yaml file in Kubernetes"

In [63]:
rag(query2)

'To apply a YAML file in Kubernetes, use the following command:\n\n```bash\nkubectl apply -f FILENAME.yaml\n```'

In [64]:
answer = rag(query2)
answer = answer.replace("\\n", "\n")

In [65]:
answer

'To apply a YAML file in Kubernetes, use the following command:\n\n```bash\nkubectl apply -f FILENAME.yaml\n```'

### Elastic Search

In [66]:
from elasticsearch import Elasticsearch

In [67]:
es_client = Elasticsearch('http://localhost:9200') 

In [69]:
index_settings = {
  "settings": {
    "analysis": {
      "analyzer": {
        "english_analyzer": {
          "type": "custom",
          "tokenizer": "standard",
          "filter": [
            "lowercase",
            "stop",
            "porter_stem"
          ]
        }
      }
    }
  },
  "mappings": {
    "properties": {
      "title": { "type": "text", "analyzer": "english_analyzer" },
      "content_type": { "type": "keyword" },
      "source_file": { "type": "keyword" },
      "weight": { "type": "integer" },
      "chunk_id": { "type": "keyword" },
      "chunk_text": { "type": "text", "analyzer": "english_analyzer" }
    }
  }
}

index_name = "k8s_docs"

es_client.indices.delete(index=index_name)
es_client.indices.create(index=index_name, body=index_settings)

/var/folders/x1/0r7y2h8j4c52sbjgqxwc9m5c0000gn/T/ipykernel_59975/150554304.py:32: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'k8s_docs'})

In [70]:
documents[0]

{'title': 'Label',
 'text': 'Tags objects with identifying attributes that are meaningful and relevant to users.\n\n<!--more--> \n\nLabels are key/value pairs that are attached to objects such as . They are used to organize and to select subsets of objects.',
 'source_file': 'label.md'}

In [71]:
from tqdm.auto import tqdm

In [72]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 229/229 [00:00<00:00, 438.57it/s]


In [73]:
def elastic_search(query):
    search_query = {
        "bool": {
            "must": [
                {
                    "multi_match": {
                        "query": query,
                        "fields": ["chunk_text^3", "title", "source_file"],
                        "type": "best_fields"
                    }
                }
            ]
        }
    }
    
    response = es_client.search(index=index_name, query=search_query, size=5)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [74]:
def rag_elastic(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [75]:
rag_elastic(query)

'To get the status of pods in Kubernetes, use the following command:\n\n```bash\nkubectl get pods\n```'

In [76]:
query3 = "What IP is assigned by default to a service?"

In [77]:
rag_elastic(query3)

'The context provided does not specify what IP is assigned by default to a service.'

In [78]:
query4 = "What does kubectl describe do?"
rag_elastic(query4)

'Show details of a specific resource or group of resources.\n\nPrint a detailed description of the selected resources, including related resources such as events or controllers. You may select a single object by name, all objects of that type, provide a name prefix, or label selector. For example:\n\n```bash\n$ kubectl describe TYPE NAME_PREFIX\n```\n\nThis command will first check for an exact match on TYPE and NAME_PREFIX. If no such resource exists, it will output details for every resource that has a name prefixed with NAME_PREFIX.\n\nUse "kubectl api-resources" for a complete list of supported resources.\n\n```bash\nkubectl describe (-f FILENAME | TYPE [NAME_PREFIX | -l label] | TYPE/NAME)\n```\n\n```bash\n# Describe a node\nkubectl describe nodes kubernetes-node-emt8.c.myproject.internal\n\n# Describe a pod\nkubectl describe pods/nginx\n\n# Describe a pod identified by type and name in "pod.json"\nkubectl describe -f pod.json\n\n# Describe all pods\nkubectl describe pods\n\n# Des